### Detectar formacion de canal Bajista - Alcista

In [1]:
import pandas as pd
import numpy as np
#import plotly.graph_objects as go
import yfinance as yf
import talib as ta
#import matplotlib.pyplot as plt
#import math
from datetime import date, timedelta
from scipy import stats
from scipy.signal import argrelextrema
#from bokeh.plotting import figure, show, column
from bokeh.models import NumeralTickFormatter, Span
from sklearn.linear_model import LinearRegression
import os
from backtesting import Backtest, Strategy

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
df_h = pd.read_csv("data/dataxh.txt", sep="\t")
tickers = [
    'SPY',
    'META',
    'AAPL',
    'AMZN',
    'NFLX',
    'MRNA',
    'TSLA',
    'TNA',
    'GLD',
    'SLV',
    'USO',
    'BAC',
    'CVX',
    'XOM',
    'QQQ',
    'MSFT',
    'NVDA',
    'WMT',
    'BA',
    'DIS',
    'CAT',
    'IBM',
    'WFC',
    'PLTR',
    'AMD',
    'AVGO',
    'HOOD',
    'CRWV',
    'MSTR',
    'UNH',
    'GOOG',
    'APP',
    'UBER'
]

#    'SPY',
#    'META',
#    'AAPL',
#    'AMZN',
#    'NFLX',
#    'MRNA',
#    'TSLA',
#    'TNA',
#    'GLD',
#    'SLV',
#    'USO',
#    'BAC',
#    'CVX',
#    'XOM',
#    'QQQ'

In [3]:
df_h['datetime'] = pd.to_datetime(df_h['datetime'])
df = pd.DataFrame()
for ticker in tickers:
    company = df_h.query("companyName==@ticker").copy()
    company.sort_values(by=['datetime'])
    #company = yf.download(ticker, start = ini2_str, end = today_str, interval='60m')
    #company.columns = [company.columns[0][0], company.columns[1][0], company.columns[2][0], company.columns[3][0], company.columns[4][0]]
    #company.rename(columns={'Datetime':'Gmt time'}, inplace = True)
    #company['datetime'] = pd.to_datetime (company.index)
    #company['companyName'] = ticker
    #company['SMA35'] = company['Close'].rolling(35).mean()
    #company.dropna(inplace=False)
    #company['SMA50'] = company['Close'].rolling(50).mean()
    #company.dropna(inplace=False)
    company['EMA35'] = company['Close'].ewm(span=35, adjust=False).mean()
    company.dropna(inplace=False)
    company['EMA50'] = company['Close'].ewm(span=50, adjust=False).mean()
    company.dropna(inplace=False)

    company['EMA25'] = company['Close'].ewm(span=25, adjust=False).mean()
    company.dropna(inplace=False)
    company['EMA40'] = company['Close'].ewm(span=40, adjust=False).mean()
    company.dropna(inplace=False)
    
    #company['SMA100'] = company['Close'].rolling(100).mean()
    #company.dropna(inplace=False)
    #company['SMA200'] = company['Close'].rolling(200).mean()
    #company.dropna(inplace=False)
    
    df = pd.concat([df, company],ignore_index=True)

### Detect Pivots/Fractals

In [4]:
ord=20
ord2=5
#for ticker in tickers:
#print("====", ticker)
#dfpl = df[(df.companyName==ticker)] #df.query("companyName=='AAPL'")
max_idx = argrelextrema(df['Close'].values, np.greater, order=ord)[0]
min_idx = argrelextrema(df['Close'].values, np.less, order=ord)[0]

max_idx2 = argrelextrema(df['Close'].values, np.greater, order=ord2)[0]
min_idx2 = argrelextrema(df['Close'].values, np.less, order=ord2)[0]
#print(max_idx)
#print(min_idx)
#df['pivotHigh'] = np.nan
#df['pivotLow'] = np.nan
#df['isPivot'] = np.nan
#df['isPivot'] = np.nan
# Aplicar el cálculo solo a los índices en la lista
df.loc[max_idx, 'pivotHigh'] = df['High']+1e-3
df.loc[min_idx, 'pivotLow'] = df['Low']-(1e-3)
df.loc[max_idx, 'isPivot'] = 1
df.loc[min_idx, 'isPivot'] = 2

df.loc[max_idx2, 'pivotHigh2'] = df['High']+1e-3
df.loc[min_idx2, 'pivotLow2'] = df['Low']-(1e-3)
df.loc[max_idx2, 'isPivot2'] = 1
df.loc[min_idx2, 'isPivot2'] = 2

#### Canales Bajistas y Alcistas
###### - Cuando se dan el cruce de medias moviles 35 y 50

In [5]:
#df = df.drop(columns=['cruce_medias']) # Drops column 'B'

In [6]:
# Señal: 1 = compra, -1 = venta, 0 = nada
#df['prev_SMA35'] = df['SMA35'].shift(1)
#df['prev_SMA50'] = df['SMA50'].shift(1)
df['prev_EMA35'] = df['EMA35'].shift(1)
df['prev_EMA50'] = df['EMA50'].shift(1)


#Cruce de medias
#df['cruce_medias'] = 0
#df.loc[(df['prev_SMA35'] < df['prev_SMA50']) & (df['SMA35'] > df['SMA50']), 'cruce_medias'] = 1  # Golden Cross (Compra)
#df.loc[(df['prev_SMA35'] > df['prev_SMA50']) & (df['SMA35'] < df['SMA50']), 'cruce_medias'] = -1 # Death Cross (Venta)

df['cruce_medias'] = 0
df.loc[(df['prev_EMA35'] < df['prev_EMA50']) & (df['EMA35'] > df['EMA50']), 'cruce_medias'] = 1  # Golden Cross (Compra)
df.loc[(df['prev_EMA35'] > df['prev_EMA50']) & (df['EMA35'] < df['EMA50']), 'cruce_medias'] = -1 # Death Cross (Venta)

In [7]:
#df["cdlengulfing"] = ta.CDLENGULFING(df['Open'], df['High'], df['Low'], df['Close'])
#df["cdlhammer"] = ta.CDLHAMMER(df['Open'], df['High'], df['Low'], df['Close'])
#df["cdlmorningstar"] = ta.CDLMORNINGSTAR(df['Open'], df['High'], df['Low'], df['Close'])
#df["cdlpiercing"] = ta.CDLPIERCING(df['Open'], df['High'], df['Low'], df['Close'])
#df["cdlclosingmarubozu"] = ta.CDLCLOSINGMARUBOZU(df['Open'], df['High'], df['Low'], df['Close'])
#df["cdlmarubozu"] = ta.CDLMARUBOZU(df['Open'], df['High'], df['Low'], df['Close'])
#df["cdl3whitesoldiers"] = ta.CDL3WHITESOLDIERS(df['Open'], df['High'], df['Low'], df['Close'])
#df["cdlharami"] = ta.CDLHARAMI(df['Open'], df['High'], df['Low'], df['Close'])
#df["cdlharamicross"] = ta.CDLHARAMICROSS(df['Open'], df['High'], df['Low'], df['Close'])
#df["cdlinvertdhammer"] = ta.CDLINVERTEDHAMMER(df['Open'], df['High'], df['Low'], df['Close'])
#df["cdlladderbottom"] = ta.CDLLADDERBOTTOM(df['Open'], df['High'], df['Low'], df['Close'])

In [8]:
#df = df.drop(columns=['ind_posicion']) # Drops column 'B'
#df = df.drop(columns=['isBreakOutIni']) # Drops column 'B'
#df = df.drop(columns=['isBreakOutFinal']) # Drops column 'B'

In [9]:
#Funcion obtner vela entrada
def obtEntrada(dfpl,i,j):
    indiceFinal=0
    indiceFinal2=0
    if (dfpl["cruce_medias"][i]==1): #ALCISTA               
        #ALZA, velas por encima de promedios moviles
        #ultimo high por encima y ultimo low cerca a los promedios                
        siguiente_H = dfpl[(dfpl.index>=j) & ((dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1))].head(1)
        if (siguiente_H.shape[0]>0):
            if (siguiente_H.iloc[0]['High']>siguiente_H.iloc[0]['EMA35']):
                indice = siguiente_H.index[0]
                siguiente_L = dfpl[(dfpl.index>indice) & ((dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2))].head(1)
                if (siguiente_L.shape[0]>0):
                    if (((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])<1) &  (siguiente_L.iloc[0]['Low']<siguiente_H.iloc[0]['High'])):
                        indiceFinal = siguiente_L.index[0]
                    else:
                        indiceFinal2 = siguiente_L.index[0]
    
    
    elif (dfpl["cruce_medias"][i]==-1): #BAJISTA                    
        #BAJA, velas por debajo de promedios moviles
        #ultimo low por debajo y ultimo high cerca a los promedios
        siguiente_L = dfpl[(dfpl.index>=j) & ((dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2)) ].head(1)
        if (siguiente_L.shape[0]>0):
            if (siguiente_L.iloc[0]['Low']<siguiente_L.iloc[0]['EMA35']):
                indice = siguiente_L.index[0]
                siguiente_H = dfpl[(dfpl.index>indice) & ((dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1))].head(1)
                if (siguiente_H.shape[0]>0):
                    if (((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])<1) & (siguiente_H.iloc[0]['High']>siguiente_L.iloc[0]['Low'])):
                        indiceFinal = siguiente_H.index[0]
                    else:
                        indiceFinal2 = siguiente_L.index[0]

    return indiceFinal, indiceFinal2

In [10]:
#Funcion revisar Velas
def revisarVelas(dfpl, indiceFinal):
    ind_trendHL=0
    ultimos_2H = dfpl[(dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1)].loc[:indiceFinal].tail(2)
    ultimos_2L = dfpl[(dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2)].loc[:indiceFinal].tail(2)

    #print("inicio")
    #print(ultimos_2H.index[0])
    #print("fin")
        
    if (ultimos_2H.shape[0]==2 & ultimos_2L.shape[0]==2):
        #print("h1")
        penultimo_valorH = ultimos_2H.iloc[0]['High']
        ultimo_valorH = ultimos_2H.iloc[1]['High']
        penultimo_valorL = ultimos_2L.iloc[0]['Low']
        ultimo_valorL = ultimos_2L.iloc[1]['Low']

        print("idpenultimoH:",  ultimos_2H.index[0], ", penultimo_valorH:", penultimo_valorH, "idultimoH:",  ultimos_2H.index[1], ", ultimo_valorH:", ultimo_valorH)
        print("idpenultimoL:",  ultimos_2L.index[0], ", penultimo_valorL:", penultimo_valorL, "idultimoH:",  ultimos_2L.index[1], ", ultimo_valorL:", ultimo_valorL)

        #En caso de tendencia ALCISTA
        #tomar los 2 ultimos HH – Higher High (Maximos mas Altos)
        #tomar los 2 ultimos HL – Higher Low (Maximos mas bajos)
        if (dfpl["cruce_medias"][i]==1): #ALCISTA
            #print("h2")
            if ((ultimo_valorH>penultimo_valorH) | (ultimo_valorL>penultimo_valorL)):
                #print("h21")
                ind_trendHL=1

        #En caso de tendencia BAJISTA
        #tomar los 2 ultimos LH – Lower High (Minimos mas Altos)
        #tomar los 2 ultimos LL – Lower Low (Minimos mas bajos)
        elif (dfpl["cruce_medias"][i]==-1): #BAJISTA
            #print("h3")
            if ((ultimo_valorH<penultimo_valorH) | (ultimo_valorL<penultimo_valorL)):
                #print("h31")
                ind_trendHL=1
    return ind_trendHL

In [11]:
#Funcion revisar SLOPE
def obtSlope(dfpl,i,j,indiceFinal):
    print("j:",j)
    ind_sl=0
    #Inicio determinar slope que indica tendencia al ALZA o a la BAJA
    medias35 = dfpl.loc[j:indiceFinal].EMA35.values
    idxmedias35 = dfpl.loc[j:indiceFinal].EMA35.index
    medias50 = dfpl.loc[j:indiceFinal].EMA50.values
    idxmedias50 = dfpl.loc[j:indiceFinal].EMA50.index
    mediasClose = dfpl.loc[j:indiceFinal].Close.values
    idxmediasClose = dfpl.loc[j:indiceFinal].Close.index
    
    if ((len(medias35)>=2) and  (len(medias50)>=2)):
        print("h1")
        sl35, interc35, r_value35, _, _ = stats.linregress(idxmedias35,medias35)
        sl50, interc50, r_value50, _, _ = stats.linregress(idxmedias50,medias50)
        sl, interc, r_value, _, _ = stats.linregress(idxmediasClose,mediasClose)
    #Fin slope

    print ("sl35:", sl35,"sl50:", sl50,"sl:", sl)
    print ("cruce_medias:", dfpl["cruce_medias"][i])
    
    if (dfpl["cruce_medias"][i]==1): #ALCISTA
        print("h2")
        #Revision de pendiente
        if (sl35>0 or sl50>0 #Pendiente positiva
           #and np.isclose(sl35, sl50, atol=1e-5) #Verificar si son paralelas
           #and interc35>interc50 #La línea 35 está por encima de la línea 50
           ):
            ind_sl=1
    elif (dfpl["cruce_medias"][i]==-1): #BAJISTA
        print("h3")
        #Revision de pendiente
        if (sl35<0 or sl50<0 #Pendiente negativa
            #and np.isclose(sl35, sl50, atol=1e-5) #Verificar si son paralelas
            #and interc50>interc35 #La línea 50 está por encima de la línea 35
           ):
            print("h4")
            ind_sl=1
    return ind_sl, sl35, sl50, sl

In [12]:
df_casos = pd.DataFrame()
caso = 0
ticker2 = ""
window=5
backeval=75
posteval=0
backevalTrend=0
trendH = []
trendL = []
ispivot=0
dfprincipal = df.copy()

for i, row in dfprincipal.iterrows():
    #if (i==424) or (i==479):
    cant = 0
    cant2 = 0
    ind_sl=0
    ind_trendHL=0
    indiceFinal = 0
    indiceFinal2 = 0

    ticker = dfprincipal['companyName'][i]
    #Reinicio de casos por company
    if ticker2 != ticker:
        ticker2 = ticker
        caso = 0

    idvelafintend=0
    tipo=""
    if (dfprincipal["cruce_medias"][i]==1): #ALCISTA
        velafintend = dfprincipal[(dfprincipal["cruce_medias"]==-1) & (dfprincipal.index>i)].head(1)
        tipo="ALZA"
        if (velafintend.shape[0]>0):
            idvelafintend = velafintend.index[0]
    elif (dfprincipal["cruce_medias"][i]==-1): #BAJISTA
        velafintend = dfprincipal[(dfprincipal["cruce_medias"]==1) & (dfprincipal.index>i)].head(1)
        tipo="BAJA" 
        if (velafintend.shape[0]>0):
            idvelafintend = velafintend.index[0]
    
    if (tipo!=""):    
        if ((idvelafintend-i)<=75):
            posteval=i+75
        elif ((idvelafintend-i)>75):
            posteval=i+idvelafintend+10
            
        
                
        #if (dfpl["cruce_medias"][i]==-1):
        #    #print("posible caso a la BAJA:",i, dfprincipal['companyName'][i])
        #    ispivot=1
        #    ispivot2=2
        #    if (sl<0):
        #        #print("h1")
        #        ind_sl=1
        #
        #elif (dfpl["cruce_medias"][i]==1):
        #    #print("posible caso a la ALZA:",i, dfprincipal['companyName'][i])
        #    ispivot=2
        #    ispivot2=1
        #    if (sl>0):
        #        #print("h2")
        #        ind_sl=1
        ind_sl=0          
    
        #Se dio el cruce de medias, antes existio un pivot high aprox 2 dias como maximo
        #fecha= dfpl['datetime'][i]
        #fecha_limite = dfpl['datetime'][i] - pd.Timedelta(days=2) #resta dias calendarios
        #fecha_limite = dfpl['datetime'][i] - pd.offsets.BDay(2) #resta dias laborables
        #print("fecha:", dfpl['datetime'][i] ,", fecha_limite:", fecha_limite)        
        #cnt1 = dfpl.query("isPivot==@ispivot and datetime>=@fecha_limite and datetime<=@fecha")["isPivot"].count()
        #cnt2 = dfpl.query("isPivot==@ispivot2 and datetime>=@fecha_limite and datetime<=@fecha")["isPivot"].count()        
        #print("cantidad:", cnt)
        #if (cnt1>0 and cnt2==0):
        #Obtener casos
        #print("h00")
        
        #Verificar hasta que la tendencia cambie, se vuelva a dar el cruce de medias 35 y 50
        #print("i:",i)
        df2 = pd.DataFrame()
        if (dfprincipal["cruce_medias"][i]==1): #ALCISTA
            if (velafintend.shape[0]>0):
                idvelafintend = velafintend.index[0]
                #print("hhh1:idvelafintend",idvelafintend,", i:", i)
                df2 = (dfprincipal.query("index>=@i and index<=@idvelafintend")).copy()
            else:
                df2 = (dfprincipal.query("index>=@i")).copy()
        elif (dfprincipal["cruce_medias"][i]==-1): #BAJISTA                
            if (velafintend.shape[0]>0):
                idvelafintend = velafintend.index[0]
                #print("hhh2")
                df2 = (dfprincipal.query("index>=@i and index<=@idvelafintend")).copy()
            else:
                df2 = (dfprincipal.query("index>=@i")).copy()
    
        iniEval = i
        
        #print("====>idvelafintend:",idvelafintend)
        #print("i:",i, ", maxid:", df2.index[-1])
        
        for j, row in df2.iterrows():             
            if (j==iniEval):      
                print("posible caso:",i, dfprincipal['companyName'][i], "==>", tipo)
                dfpl = pd.DataFrame()
                dfpl = (df[(df.companyName==ticker)].loc[i-backeval:posteval]).copy()
                print ("ini i:", i)
                print ("oportunidad:", j)
                id2=0
                indiceFinal, indiceFinal2 = obtEntrada(dfpl, i, j)
                print ("isBreakOutIni:", indiceFinal)
                if (indiceFinal>0):
                    ind_trendHL = revisarVelas(dfpl, indiceFinal)
                #print("indiceFinal:",indiceFinal,",ind_trendHL:",ind_trendHL)    
                if (indiceFinal>0):
                    ind_sl, sl35, sl50, sl = obtSlope(dfpl,i,j,indiceFinal)
        
                #print("sl35:",sl35,"sl50:",sl50)
                print("==>indiceFinal:",indiceFinal,"ind_trendHL:",ind_trendHL, ", ind_sl:", ind_sl)
    
                
                
                if ((indiceFinal>0) and (ind_trendHL>0) and (ind_sl>0)):
                    #INSERT CASO
                    print("insert caso")
    
                    #Final de caso es siguiente Pivot
                    if (dfpl["cruce_medias"][i]==1): #ALCISTA
                        dfpl.loc[i,'ind_posicion']=1
                        dfpl.loc[indiceFinal,'isBreakOutIni']=1
                        k=0
                        while (k<=3):
                            cnt2 = dfpl.query("index>@indiceFinal and isPivot==1").shape[0]
                            if cnt2>0:        
                                id2 = dfpl.query("index>@indiceFinal and isPivot==1").index[0]
                                dfpl.loc[id2,'isBreakOutFinal'] = 1
                                k=4
                            else:
                                idfinal = dfpl.index[-1] 
                                idfinal2 = idfinal+25
                                if idfinal2 in dfprincipal[dfprincipal['companyName']==ticker].index:
                                    dfpl2 = (dfprincipal[(dfprincipal.companyName==ticker)].loc[idfinal+1:idfinal2]).copy()
                                    dfpl = pd.concat([dfpl, dfpl2],ignore_index=False)                           
                                else:                  
                                    dfpl['isBreakOutFinal'] = np.nan
                                k=k+1
                               
                        print ("isBreakOutFinal:", id2)
                            
                    elif (dfpl["cruce_medias"][i]==-1): #BAJISTA
                        dfpl.loc[i,'ind_posicion']=-1
                        dfpl.loc[indiceFinal,'isBreakOutIni']=-1
                        k=0
                        while (k<=3):
                            cnt2 = dfpl.query("index>@indiceFinal and isPivot==2").shape[0]
                            if cnt2>0:        
                                id2 = dfpl.query("index>@indiceFinal and isPivot==2").index[0]
                                dfpl.loc[id2,'isBreakOutFinal'] = -1
                                k=4
                            else:
                                idfinal = dfpl.index[-1] 
                                idfinal2 = idfinal+25
                                if idfinal2 in dfprincipal[dfprincipal['companyName']==ticker].index:
                                    dfpl2 = (dfprincipal[(dfprincipal.companyName==ticker)].loc[idfinal+1:idfinal2]).copy()
                                    dfpl = pd.concat([dfpl, dfpl2],ignore_index=False)                           
                                else:                  
                                    dfpl['isBreakOutFinal'] = np.nan
                                k=k+1
                    
                    if (dfprincipal["cruce_medias"][i]==-1):                    
                        dfprincipal.loc[i,'ind_posicion']=-1                    
                        dfprincipal.loc[indiceFinal,'isBreakOutIni']=-1
                        if id2!=0:
                            dfprincipal.loc[id2,'isBreakOutFinal']=-1
                    elif (dfprincipal["cruce_medias"][i]==1):                        
                        dfprincipal.loc[i,'ind_posicion']=1                   
                        dfprincipal.loc[indiceFinal,'isBreakOutIni']=1
                        if id2!=0:
                            dfprincipal.loc[id2,'isBreakOutFinal']=1
        
                    #print("h5")                
                    caso=caso+1
                    dfpl["caso"] = caso
                    #dfprincipal.loc[i,'caso']=caso
                    print(i, dfprincipal['companyName'][i], ", j:", j, ", caso:", caso , "cruce medias:",dfprincipal["cruce_medias"][i],", slope35:", sl35, ", slope50:", sl50, ", slope:", sl)
                    if len(df_casos)<1:
                        df_casos = dfpl.copy()
                    else:
                        df_casos = pd.concat([df_casos, dfpl],ignore_index=False)
        
                    if (id2!=0):
                        iniEval = id2
                    else:
                        iniEval = indiceFinal2

posible caso: 118 SPY ==> BAJA
ini i: 118
oportunidad: 118
isBreakOutIni: 128
idpenultimoH: 110 , penultimo_valorH: 456.989990234375 idultimoH: 128 , ultimo_valorH: 452.510009765625
idpenultimoL: 90 , penultimo_valorL: 451.5499877929688 idultimoH: 119 , ultimo_valorL: 447.4088134765625
j: 118
h1
sl35: -0.182621792685983 sl50: -0.14137520427431594 sl: 0.25958002263849794
cruce_medias: -1
h3
h4
==>indiceFinal: 128 ind_trendHL: 1 , ind_sl: 1
insert caso
118 SPY , j: 118 , caso: 1 cruce medias: -1 , slope35: -0.182621792685983 , slope50: -0.14137520427431594 , slope: 0.25958002263849794
posible caso: 118 SPY ==> BAJA
ini i: 118
oportunidad: 196
isBreakOutIni: 210
idpenultimoH: 174 , penultimo_valorH: 448.1099853515625 idultimoH: 210 , ultimo_valorH: 441.1799926757813
idpenultimoL: 162 , penultimo_valorL: 444.4389953613281 idultimoH: 196 , ultimo_valorL: 433.010009765625
j: 196
h1
sl35: -0.18886928304993658 sl50: -0.1930741722808656 sl: 0.36096300397600506
cruce_medias: -1
h3
h4
==>indiceFi

posible caso: 334 SPY ==> BAJA
ini i: 334
oportunidad: 417
isBreakOutIni: 426
idpenultimoH: 407 , penultimo_valorH: 428.4700012207031 idultimoH: 426 , ultimo_valorH: 425.4299011230469
idpenultimoL: 388 , penultimo_valorL: 422.760009765625 idultimoH: 417 , ultimo_valorL: 420.1799926757813
j: 417
h1
sl35: -0.16670776350324526 sl50: -0.18071890925460993 sl: 0.4572154651988678
cruce_medias: -1
h3
h4
==>indiceFinal: 426 ind_trendHL: 1 , ind_sl: 1
insert caso
334 SPY , j: 417 , caso: 8 cruce medias: -1 , slope35: -0.16670776350324526 , slope50: -0.18071890925460993 , slope: 0.4572154651988678
posible caso: 448 SPY ==> ALZA
ini i: 448
oportunidad: 448
isBreakOutIni: 0
==>indiceFinal: 0 ind_trendHL: 0 , ind_sl: 0
posible caso: 502 SPY ==> BAJA
ini i: 502
oportunidad: 502
isBreakOutIni: 0
==>indiceFinal: 0 ind_trendHL: 0 , ind_sl: 0
posible caso: 570 SPY ==> ALZA
ini i: 570
oportunidad: 570
isBreakOutIni: 0
==>indiceFinal: 0 ind_trendHL: 0 , ind_sl: 0
posible caso: 855 SPY ==> BAJA
ini i: 855
o

posible caso: 884 SPY ==> ALZA
ini i: 884
oportunidad: 1165
isBreakOutIni: 1172
idpenultimoH: 1142 , penultimo_valorH: 514.2000122070312 idultimoH: 1165 , ultimo_valorH: 518.2200927734375
idpenultimoL: 1156 , penultimo_valorL: 508.5700073242188 idultimoH: 1172 , ultimo_valorL: 508.5
j: 1165
h1
sl35: 0.04382936586195691 sl50: 0.07254815986946046 sl: -1.0077994210379444
cruce_medias: 1
h2
==>indiceFinal: 1172 ind_trendHL: 1 , ind_sl: 1
insert caso
isBreakOutFinal: 1189
884 SPY , j: 1165 , caso: 14 cruce medias: 1 , slope35: 0.04382936586195691 , slope50: 0.07254815986946046 , slope: -1.0077994210379444
posible caso: 884 SPY ==> ALZA
ini i: 884
oportunidad: 1189
isBreakOutIni: 1202
idpenultimoH: 1189 , penultimo_valorH: 517.2100830078125 idultimoH: 1199 , ultimo_valorH: 514.97998046875
idpenultimoL: 1172 , penultimo_valorL: 508.5 idultimoH: 1202 , ultimo_valorL: 508.9299926757813
j: 1189
h1
sl35: -0.011727228191695688 sl50: 0.023975090448943075 sl: -0.5138459928743119
cruce_medias: 1
h2
=

posible caso: 1567 SPY ==> BAJA
ini i: 1567
oportunidad: 1567
isBreakOutIni: 1578
idpenultimoH: 1545 , penultimo_valorH: 530.0540161132812 idultimoH: 1578 , ultimo_valorH: 529.3099975585938
idpenultimoL: 1541 , penultimo_valorL: 524.719970703125 idultimoH: 1573 , ultimo_valorL: 518.3599853515625
j: 1567
h1
sl35: -0.23751910259959005 sl50: -0.18236966096729118 sl: 0.1551938357053103
cruce_medias: -1
h3
h4
==>indiceFinal: 1578 ind_trendHL: 1 , ind_sl: 1
insert caso
1567 SPY , j: 1567 , caso: 19 cruce medias: -1 , slope35: -0.23751910259959005 , slope50: -0.18236966096729118 , slope: 0.1551938357053103
posible caso: 1593 SPY ==> ALZA
ini i: 1593
oportunidad: 1593
isBreakOutIni: 0
==>indiceFinal: 0 ind_trendHL: 0 , ind_sl: 0
posible caso: 1799 SPY ==> BAJA
ini i: 1799
oportunidad: 1799
isBreakOutIni: 1813
idpenultimoH: 1791 , penultimo_valorH: 559.52001953125 idultimoH: 1813 , ultimo_valorH: 556.7349853515625
idpenultimoL: 1776 , penultimo_valorL: 559.6300048828125 idultimoH: 1801 , ultimo

posible caso: 2070 SPY ==> ALZA
ini i: 2070
oportunidad: 2070
isBreakOutIni: 0
==>indiceFinal: 0 ind_trendHL: 0 , ind_sl: 0
posible caso: 2279 SPY ==> BAJA
ini i: 2279
oportunidad: 2279
isBreakOutIni: 2303
idpenultimoH: 2281 , penultimo_valorH: 584.4600219726562 idultimoH: 2303 , ultimo_valorH: 582.9600219726562
idpenultimoL: 2287 , penultimo_valorL: 578.5399780273438 idultimoH: 2295 , ultimo_valorL: 578.4299926757812
j: 2279
h1
sl35: 0.030699642373898717 sl50: 0.02476834683525882 sl: 0.06422884427584134
cruce_medias: -1
h3
==>indiceFinal: 2303 ind_trendHL: 1 , ind_sl: 0
posible caso: 2281 SPY ==> ALZA
ini i: 2281
oportunidad: 2281
isBreakOutIni: 2287
idpenultimoH: 2264 , penultimo_valorH: 584.5 idultimoH: 2281 , ultimo_valorH: 584.4600219726562
idpenultimoL: 2271 , penultimo_valorL: 574.4149780273438 idultimoH: 2287 , ultimo_valorL: 578.5399780273438
j: 2281
h1
sl35: -0.0020676208590616947 sl50: 0.001444228175269505 sl: -0.868924822126116
cruce_medias: 1
h2
==>indiceFinal: 2287 ind_tr

...

posible caso: 111835 UBER ==> BAJA
ini i: 111835
oportunidad: 111835
isBreakOutIni: 0
==>indiceFinal: 0 ind_trendHL: 0 , ind_sl: 0
posible caso: 111962 UBER ==> ALZA
ini i: 111962
oportunidad: 111962
isBreakOutIni: 111986
idpenultimoH: 111970 , penultimo_valorH: 67.3499984741211 idultimoH: 111977 , ultimo_valorH: 68.83999633789062
idpenultimoL: 111955 , penultimo_valorL: 60.16999816894531 idultimoH: 111986 , ultimo_valorL: 64.16999816894531
j: 111962
h1
sl35: 0.14700044301124995 sl50: 0.12075254417365491 sl: 0.06242493262657752
cruce_medias: 1
h2
==>indiceFinal: 111986 ind_trendHL: 1 , ind_sl: 1
insert caso
isBreakOutFinal: 112022
111962 UBER , j: 111962 , caso: 34 cruce medias: 1 , slope35: 0.14700044301124995 , slope50: 0.12075254417365491 , slope: 0.06242493262657752
posible caso: 111962 UBER ==> ALZA
ini i: 111962
oportunidad: 112022
isBreakOutIni: 112033
idpenultimoH: 112001 , penultimo_valorH: 66.48999786376953 idultimoH: 112022 , ultimo_valorH: 69.54499816894531
idpenultimoL: 11

posible caso: 112084 UBER ==> BAJA
ini i: 112084
oportunidad: 112084
isBreakOutIni: 112089
idpenultimoH: 112070 , penultimo_valorH: 69.67520141601562 idultimoH: 112089 , ultimo_valorH: 67.84860229492188
idpenultimoL: 112066 , penultimo_valorL: 68.2300033569336 idultimoH: 112084 , ultimo_valorL: 65.30000305175781
j: 112084
h1
sl35: -0.04741811657653199 sl50: -0.03613641124932364 sl: 0.3149939400809152
cruce_medias: -1
h3
h4
==>indiceFinal: 112089 ind_trendHL: 1 , ind_sl: 1
insert caso
112084 UBER , j: 112084 , caso: 37 cruce medias: -1 , slope35: -0.04741811657653199 , slope50: -0.03613641124932364 , slope: 0.3149939400809152
posible caso: 112104 UBER ==> ALZA
ini i: 112104
oportunidad: 112104
isBreakOutIni: 112114
idpenultimoH: 112097 , penultimo_valorH: 68.8499984741211 idultimoH: 112107 , ultimo_valorH: 70.19000244140625
idpenultimoL: 112102 , penultimo_valorL: 67.30000305175781 idultimoH: 112114 , ultimo_valorL: 63.54999923706055
j: 112104
h1
sl35: -0.04065066545611222 sl50: -0.0255

ini i: 112368
oportunidad: 112368
isBreakOutIni: 112375
idpenultimoH: 112355 , penultimo_valorH: 75.4800033569336 idultimoH: 112375 , ultimo_valorH: 73.30000305175781
idpenultimoL: 112350 , penultimo_valorL: 73.2249984741211 idultimoH: 112369 , ultimo_valorL: 70.5
j: 112368
h1
sl35: -0.08376598284795429 sl50: -0.06316299177007914 sl: 0.13160305931454613
cruce_medias: -1
h3
h4
==>indiceFinal: 112375 ind_trendHL: 1 , ind_sl: 1
insert caso
112368 UBER , j: 112368 , caso: 39 cruce medias: -1 , slope35: -0.08376598284795429 , slope50: -0.06316299177007914 , slope: 0.13160305931454613
posible caso: 112368 UBER ==> BAJA
ini i: 112368
oportunidad: 112405
isBreakOutIni: 112411
idpenultimoH: 112386 , penultimo_valorH: 75.41000366210938 idultimoH: 112411 , ultimo_valorH: 70.06999969482422
idpenultimoL: 112376 , penultimo_valorL: 70.83000183105469 idultimoH: 112405 , ultimo_valorL: 62.7599983215332
j: 112405
h1
sl35: -0.1728228882286891 sl50: -0.16398337409987918 sl: 0.693843024117606
cruce_medias

ini i: 112438
oportunidad: 112464
isBreakOutIni: 112470
idpenultimoH: 112454 , penultimo_valorH: 74.80000305175781 idultimoH: 112464 , ultimo_valorH: 75.80000305175781
idpenultimoL: 112458 , penultimo_valorL: 72.05180358886719 idultimoH: 112470 , ultimo_valorL: 71.4000015258789
j: 112464
h1
sl35: 0.02681554404734194 sl50: 0.042604857370200984 sl: -0.6428576878138951
cruce_medias: 1
h2
==>indiceFinal: 112470 ind_trendHL: 1 , ind_sl: 1
insert caso
isBreakOutFinal: 112549
112438 UBER , j: 112464 , caso: 42 cruce medias: 1 , slope35: 0.02681554404734194 , slope50: 0.042604857370200984 , slope: -0.6428576878138951
posible caso: 112438 UBER ==> ALZA
ini i: 112438
oportunidad: 112549
isBreakOutIni: 112564
idpenultimoH: 112540 , penultimo_valorH: 86.4800033569336 idultimoH: 112549 , ultimo_valorH: 86.44000244140625
idpenultimoL: 112529 , penultimo_valorL: 80.7300033569336 idultimoH: 112564 , ultimo_valorL: 82.16999816894531
j: 112549
h1
sl35: -0.0009840821287485698 sl50: 0.0377750333359219 sl:

posible caso: 112710 UBER ==> ALZA
ini i: 112710
oportunidad: 112710
isBreakOutIni: 112738
idpenultimoH: 112709 , penultimo_valorH: 87.79000091552734 idultimoH: 112719 , ultimo_valorH: 87.5999984741211
idpenultimoL: 112713 , penultimo_valorL: 85.5999984741211 idultimoH: 112738 , ultimo_valorL: 83.41999816894531
j: 112710
h1
sl35: 0.004983424365224447 sl50: 0.007669592479254106 sl: -0.10241257991696813
cruce_medias: 1
h2
==>indiceFinal: 112738 ind_trendHL: 0 , ind_sl: 1
posible caso: 112736 UBER ==> BAJA
ini i: 112736
oportunidad: 112736
isBreakOutIni: 112742
idpenultimoH: 112719 , penultimo_valorH: 87.5999984741211 idultimoH: 112742 , ultimo_valorH: 85.95999908447266
idpenultimoL: 112713 , penultimo_valorL: 85.5999984741211 idultimoH: 112738 , ultimo_valorL: 83.41999816894531
j: 112736
h1
sl35: -0.044041763918897914 sl50: -0.03377190633419553 sl: 0.3336500440325056
cruce_medias: -1
h3
h4
==>indiceFinal: 112742 ind_trendHL: 1 , ind_sl: 1
insert caso
112736 UBER , j: 112736 , caso: 45 cr

In [13]:
#descarga de informacion, estrategia: RUPTURA DEL CANAL BAJISTA
import os
#path = r'C:\Users\carlo\OneDrive\Documentos\TRADER\traderapp\data\rcb_h.txt'
path = 'data/cba_h.txt'
# check whether the file exists
if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
    df.to_csv(path, header=True, index=None, sep='\t', mode='w')
    #f.write(df_string)

#### BACKTESTING
###### Usando paquete backtesting.py

In [14]:
df_backtesting=dfprincipal.copy()
df_backtesting['isBreakOutIni'] = df_backtesting['isBreakOutIni'].shift(-1)
df_backtesting['isBreakOutFinal'] = df_backtesting['isBreakOutFinal'].shift(-1)
df_backtesting.set_index('datetime', inplace=True)

In [15]:
listruptura = [1,-1]

In [16]:
df_casos2 = df_casos.query("isBreakOutIni==@listruptura or isBreakOutFinal==@listruptura").copy()

In [17]:
df_casos2['Tag'] = np.where((df_casos2['isBreakOutIni']==-1) | (df_casos2['isBreakOutFinal']==-1), 'short', 'long')

In [18]:
ini = df_casos2[(df_casos2['isBreakOutIni'].isin(listruptura))].copy()
df_casos3_ini=ini.rename(columns={'Open': 'EntryPrice', 'datetime': 'EntryTime', 'companyName': 'Ticker'})[['EntryPrice','Ticker','caso','EntryTime','Tag']]
final = df_casos2[(df_casos2['isBreakOutFinal'].isin(listruptura))].copy()
df_casos3_final=final.rename(columns={'Open': 'ExitPrice', 'datetime': 'ExitTime', 'companyName': 'Ticker'})[['ExitPrice','Ticker','caso','ExitTime','Tag']]

In [19]:
df_casos3=df_casos3_ini.merge(df_casos3_final,on=['Ticker','caso','Tag'],how='outer')

In [20]:
class StrategyCanalBajistaAlcista(Strategy):
    def init(self):
        self.breakout_entry = self.I(lambda x: x, self.data.isBreakOutIni) #Indica la señal entrada al backtesting
        self.breakout_exit = self.I(lambda x: x, self.data.isBreakOutFinal) #Indica la señal salida al backtesting
    def next(self):
        #print(self.data)
        if self.breakout_entry[-1] == 1:
            self.position.close()
            self.buy(tag="long")  #COMPRA
        elif self.breakout_entry[-1]==-1:
            self.position.close()
            self.sell(tag="short") #VENTA
        elif self.breakout_exit[-1] == 1 and self.position:
            self.position.close() #CIERRE
        elif self.breakout_exit[-1] == -1 and self.position:
            self.position.close() #CIERRE

In [21]:
my_stats = {}
estadisticas = pd.DataFrame()          # lista para ticker
tradesprev = pd.DataFrame()     # lista para cada caso por ticker
for ticker in df_backtesting['companyName'].unique():
    ticker_data = df_backtesting[df_backtesting['companyName'] == ticker].copy() #Filtro por cada ticker
    bt = Backtest(ticker_data, StrategyCanalBajistaAlcista, cash=10_000)
    my_stats = bt.run()
    listTrades = my_stats['_trades']
    tradesdf = pd.DataFrame(listTrades) #trades
    tradesdf['Ticker'] = ticker
    stats_filter={
        'Ticker':ticker,
        'EntryTime':my_stats['Start'],
        'ExitTime':my_stats['End'],
        'Return [%]': my_stats['Return [%]'],
        'CAGR [%]': my_stats['Return (Ann.) [%]'],
        'Sharpe Ratio': my_stats['Sharpe Ratio'],
        'Max. Drawdown [%]': my_stats['Max. Drawdown [%]'],
        'Win Rate [%]': my_stats['Win Rate [%]'],
        '# Trades': my_stats['# Trades'],
        'Expectancy [%]': my_stats['Expectancy [%]'],
        'Profit Factor': my_stats['Profit Factor'],
        'Duration': my_stats['Duration'],
        'Avg. Trade [%]':my_stats['Avg. Trade [%]'],
        'Max. Trade Duration':my_stats['Max. Trade Duration'],
        'Avg. Trade Duration':my_stats['Avg. Trade Duration']
    }
    stat=pd.DataFrame([stats_filter])
    if tradesprev.shape[0]==0:
        tradesprev = tradesdf.copy()
        estadisticas=stat.copy()
    else:
        tradesprev = pd.concat([tradesprev, tradesdf],ignore_index=True)
        estadisticas=pd.concat([estadisticas, stat],ignore_index=True)
    # Mostramos
    #print(f"Resultados para {ticker}:")
    #print(stats)   
    ##plt.title(f"Backtest -  {ticker}")
    #bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")

Backtest.run:   0%|          | 0/3318 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3372 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3464 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3397 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3315 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/3377 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3035 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3279 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3140 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3328 [00:00<?, ?bar/s]

In [22]:
trades = pd.merge(df_casos3, tradesprev, on = ['EntryTime','Ticker','Tag'], how='left')

In [23]:
def exitPrice(x):
    if pd.isna(x['ExitPrice_x']):
        return x['ExitPrice_y']
    else:
        return x['ExitPrice_x']
trades['ExitPrice'] = trades.apply(lambda row: exitPrice(row), axis=1)

In [24]:
trades = trades.rename(columns={'EntryPrice_x': 'EntryPrice'})
trades = trades.drop(['EntryPrice_y','ExitPrice_x', 'ExitPrice_y'], axis=1)

In [25]:
def exitTime(x):
    if pd.isna(x['ExitTime_x']):
        return x['ExitTime_y']
    else:
        return x['ExitTime_x']
trades['ExitTime'] = trades.apply(lambda row: exitTime(row), axis=1)

In [26]:
def duration(x):
    if pd.isna(x['Duration']):
        if pd.isna(x['ExitTime']):
            return pd.NaT
        else:
            return x['ExitTime']-x['EntryTime']
    else:
        return x['Duration']
trades['Duration'] = trades.apply(lambda row: duration(row), axis=1)

In [27]:
trades = trades.drop(['ExitTime_x', 'ExitTime_y'], axis=1)

In [28]:
trades=trades[['Ticker','EntryTime','ExitTime','EntryPrice','ExitPrice','Duration','Size','EntryBar','ExitBar','ReturnPct','PnL','caso','Tag']]

In [29]:
for item in estadisticas['Ticker'].unique():
    count = trades[trades["Ticker"] == item].shape[0]
    estadisticas.loc[estadisticas['Ticker'] == item, '# Trades'] = count

In [30]:
#EXPORTACION DE DATAFRAME
path = 'data/backtesting'

# Asegurarse de que el directorio existe
os.makedirs(path, exist_ok=True)

path_trades = os.path.join(path, 'trades_cba.txt')
path_estadisticas = os.path.join(path, 'estadisticas_cba.txt')

for file_path in [path_trades, path_estadisticas]:
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print(f"File does not exist: {file_path}. It will be created.")

trades.to_csv(path_trades, header=True, index=False, sep='\t', mode='w')
estadisticas.to_csv(path_estadisticas, header=True, index=False, sep='\t', mode='w')

In [31]:
#trades[trades['Ticker']=="SPY"].shape[0]
#estadisticas